# NE 250 – Homework 1
###### 9/22/2017

## Problem 1

The number of molecules, $N$, found in a sample of a compound with mass $M$ is 
$$ N(\cdot) = \frac{M(\cdot) N_A}{m(\cdot)} $$
where $m$ is the molar mass of the compound, and $N_A$ is Avogadro's number ($N_A = 6.022 \times 10^{23}$ molecules per mole). Since each actinide isotope is found only once in a molecule of its respective oxide, $N$ also gives the number of atoms of an isotope in the sample. 

In [1]:
def molecule_count(M,m):
    N_A = 6.022e23
    N = M*N_A/m
    return N

To find $N$, we begin by finding the masses of the compounds in the fuel. We can decompose the total mass of the fuel, $M_f$, into its mixed oxide components:
$$ M_f = M(\text{UO}_2) + M(\text{PuO}_2). $$

Given a weight percent for plutonium, $w_{\text{P}}$, we note that $w_{\text{U}} = 1-w_{\text{P}}$ and so calculate the total masses of both the UO$_2$ and the PuO$_2$ to be
$$ M(\text{UO}_2) = (1-w_{\text{P}})M_f	$$
$$ M(\text{PuO}_2) = w_{\text{P}}M_f. $$

In [2]:
w_p = 0.3

In [3]:
def mass_oxide(w,M_f):
    M_oxide = w*M_f
    return M_oxide

The mass of the oxide component for a given isotope is given by 
$$ M(^{i}\text{XO}_2) = w_i M(\text{XO}_2) $$ 
where $w_i$ is the weight percent of the oxide of isotope $^{i}X$ out of element $X$.

In [4]:
def mass_isotope_oxide(w_i,M_oxide):
    M_i_oxide = w_i*M_oxide
    return M_i_oxide

In [5]:
# Provided weight percents
w = {'U238': 1, # all U is U238
     'Pu239': 0.697,
     'Pu240': 0.218,
     'Pu241': 0.058,
     'Pu242': 0.027
    }

Next, we must determine the molar masses of the various oxides. In general,
$$ m(^{i}\text{XO}_2) = m(^{i}\text{X}) + 2m(\text{O}), $$

In [6]:
def molar_mass_isotope_oxide(m_i,m_o):
    m_i_oxide = m_i + 2*m_o
    return m_i_oxide

Finally, we use both the total mass of a compound with its molar mass in the original formula, using provided or tabulated values:

In [7]:
# Tabulated molar masses [g/mol]
m = {'U238':  238.051,
     'Pu239': 239.052,
     'Pu240': 240.054,
     'Pu241': 241.057,
     'Pu242': 242.059,
     'O16': 15.995
    }

In [8]:
# Assume 1 gram of total fuel
M_f = 1

isotopes = ['U238','Pu239','Pu240','Pu241','Pu242']
for i in isotopes:
    M_i_ox = mass_isotope_oxide(w[i],mass_oxide(w_p,M_f))
    m_i_ox = molar_mass_isotope_oxide(m[i],m['O16'])
    N = molecule_count(M_i_ox,m_i_ox)
    print(i,': ',N)

U238 :  6.690095207764747e+20
Pu239 :  4.645775193512445e+20
Pu240 :  1.4477025775242242e+20
Pu241 :  3.837537127307754e+19
Pu242 :  1.7799079726618233e+19


## Problem 2

The mean free path of a particle is given by the formula
$$ \lambda = \frac{1}{\Sigma_t} $$

when scattering is considered to be isotropic. The macroscopic scattering cross section can be reexpressed in terms of the number density of the material, $n$, and the microscopic cross section, $\sigma_t$. For mixtures, the macroscopic cross section is the sum of the macroscopic cross sections of its components.
$$ \Sigma_{t,\text{mix}} = \sum_i \Sigma_{t,i} = \sum_i n_i \sigma_i $$

Additionally, the number density of the material can be found from the material density, $\rho$, the molar mass of the material, $m$, and Avogadro's number.
$$ n = \frac{\rho_i N_A}{m_i} $$

Then the mean free path is
$$ \lambda = \frac{1}{\sum_i \frac{\rho_i N_A \sigma_{t,i}}{m_i}} $$

In [118]:
def mfp(energy,medium,xs,rho,m):
    # Mean free path [cm]
    return 1/macroXS_mix(energy,medium,rho,m,xs)

def macroXS_mix(energy,medium,rho,m,xs):
    # Macroscopic cross section of a mixture of isotopes [cm^2]
    Sig_t_mix = 0
    if energy == '14MeV': E = 0   
    elif energy == '1MeV': E = 1
    elif energy == '0.05eV': E = 2
    for isotope in rho:
        Sig_t_mix += macroXS(rho[isotope],m[isotope],xs[isotope][E])
    return Sig_t_mix

def macroXS(rho_i,m_i,xs_i):
    # Macroscopic cross section of an isotope [cm^2]
    N_A = 6.022e23
    Sig_t = rho_i*N_A*xs_i*1e-24/m_i
    return Sig_t

### Densities

#### Air

Air is 75.5% nitrogen, 23.1% oxygen, and 1.3% argon by mass according to [NIST](https://physics.nist.gov/cgi-bin/Star/compos.pl?matno=104). We will assume that the nitrogen is entirely $^{14}$N, oxygen is entirely $^{16}$O, and $^{40}$Ar, which comprise more than 99.5% of their respective element naturally.

In [119]:
# Density [g/cm^3]
rho_air = 1.20e-3
rho_a = {'N14': 0.755*rho_air,
         'O16': 0.231*rho_air,
         'Ar40': 0.013*rho_air
        }

#### Water
Water is 11.2% hydrogen and 88.8% oxygen by mass. We are assuming the hydrogen is entirely $^{1}$H and the oxygen is entirely $^{16}$O.

In [120]:
# Density [g/cm^3]
rho_water = 1
rho_w = {'H1': 0.112*rho_water,
         'O16': 0.888*rho_water
        }

#### Uranium

Natural uranium is 99.3% U238 and 0.7% U235 by mass according to the [World Nuclear Association](http://www.world-nuclear.org/information-library/nuclear-fuel-cycle/conversion-enrichment-and-fabrication/uranium-enrichment.aspx). 

In [121]:
# Density [g/cm^3]
rho_uranium = 19.1
rho_u = {'U235': 0.007*rho_uranium,
         'U238': 0.993*rho_uranium
        }

### Molar Masses

In [122]:
# Molar mass [g/mol]
m = {'H1': 1.008,
     'N14': 14.003,
     'O16': 15.995,
     'Ar40': 39.962,
     'U235': 235.044,
     'U238': 238.051
    }

### Cross Sections

Microscopic cross sections are from ENDF/B-VII.1, found using [KAERI](http://atom.kaeri.re.kr/nuchart/)

In [123]:
# Cross sections at 14 MeV, 1 MeV, and 0.05 eV [b]
xs = {'H1': (0.687,4.246,20.673),
          'N14': (1.569,2.345,11.264),
          'O16': (1.592,8.154,3.852),
          'Ar40': (2.290,3.135,1.116),
          'U235': (5.831,6.800,472.885),
          'U238': (5.858,7.068,9.862)
         }

### Calculations

In [128]:
def loop_media(energy):
    print('Mean Free Path in')
    for medium in ['air','water','uranium']:
        if medium == 'air': rho = rho_a
        elif medium == 'water': rho = rho_w
        elif medium == 'uranium': rho = rho_u
        print('\t{}\t{} cm'.format(medium,round(mfp(energy,medium,xs,rho,m),2)))

***a) 14 MeV neutrons in air, water, and uranium***

In [130]:
energy = '14MeV'
loop_media(energy)                                

Mean Free Path in
	air	12773.78 cm
	water	10.08 cm
	uranium	3.53 cm


***b) 1 MeV neutrons in air, water, and uranium***

In [131]:
energy = '1MeV'
loop_media(energy)                                

Mean Free Path in
	air	5643.26 cm
	water	1.8 cm
	uranium	2.93 cm


***c) 0.05 eV Neutrons in air, water, and uranium***

In [132]:
energy = '0.05eV'
loop_media(energy)                                

Mean Free Path in
	air	2086.21 cm
	water	0.66 cm
	uranium	1.57 cm
